In [144]:
import numpy as np
import os, shutil
%matplotlib qt
import matplotlib.pyplot as pl
import pickle
import time

In [145]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [146]:
from evolution_utils import evenlayer, oddlayer, weak_measurement_layer, state_correlation, state_entropy, scramble
import unitary_sampler

In [147]:
# Don't change these
L_max = 60
T_max = 300

In [148]:
list(range(4,6,1))

[4, 5]

In [149]:
def get_data_ancilla(state,L,L_A,T,t_scr,theta,evolution_type,scrambling_type,seed_scram,seed_unitary,seed_outcomes,BC='PBC'):
    assert scrambling_type in ['haar','matchgate'], print('scrambling_type can only be either \'haar\' or \'matchgate\'')
    assert evolution_type in ['haar','matchgate'], print('evolution_type can only be either \'haar\' or \'matchgate\'')

    assert T<T_max

    state = scramble(state,L,t_scr,seed_scram=seed_scram,which_U=scrambling_type)

    unitary_gates = unitary_sampler.get_U_1_unitary_circuit(seed_unitary=seed_unitary,number_of_gates=int(L_max*T_max),which_U=evolution_type)
    
    cache = {}

    outcome_rng = np.random.default_rng(seed=seed_outcomes)
    total_N_m = 0

    entropy_data = []
    correlation_data = []
    ancilla_interval = list(range(L,L+L_A))
    for t in range(T):
        if t%2 == 0:
            state = evenlayer(state,unitary_gates[L_max*t: L_max*t + L//2],L=L)
        else:
            state = oddlayer(state,unitary_gates[L_max*t: L_max*t + L//2],L=L,BC=BC)
        
        m_locations = list(range(L)) # locations of weak measurement
        total_N_m += L
        state = weak_measurement_layer(state,theta,L,rng_outcome=outcome_rng,m_locations=m_locations)

        entropy_data.append(state_entropy(state,interval=ancilla_interval,renyi_index=1))
        correlation_data.append(state_correlation(state,L))
    
    cache['total_N_m'] = total_N_m
    cache['m_locations'] = m_locations
    
    return state, entropy_data, correlation_data, cache

In [275]:
def get_filename(t_scr,evolution_U_type,scram_U_type,root_direc):
    file_dir = root_direc

    if t_scr==0:
        file_dir = file_dir + 'without_scrambling' 
    else:
        if scram_U_type == 'haar':
            file_dir = file_dir + 'haar_scrambling' 
        elif scram_U_type == 'matchgate':
            file_dir = file_dir + 'matchgate_scrambling' 
    if evolution_U_type == 'haar':
        file_dir = file_dir + '/haar_evolution'
    if evolution_U_type == 'matchgate':
        file_dir = file_dir + '/matchgate_evolution'
    if not os.path.isdir(file_dir):
        os.makedirs(file_dir)

    return file_dir

In [231]:
# Fixed parameters
L_A = 1
theta_list = np.linspace(0.1,0.8,8)
initial_state = False
scram_depth = 1
depth = 2
scrambling_type = 'matchgate'
evolution_type = 'matchgate'
root_direc = 'data_weak_measurement/'
seed_unitary = 1
seed_scrambling = 1000*seed_unitary
shots = 300
BC = 'PBC'

for theta in theta_list:
    theta = np.round(theta,3)
    for L in [6,8,10,12,14][:]:
        start = time.time()
        entropy_data = []
        correlation_data = []
        for _ in range(shots):
            
            #define parameters
            if initial_state is False:
                state = np.zeros((2,)*(L+L_A))
                index_state_0 = (1,0)*(L//2) + (0,)
                index_state_1 = (1,0)*(L//2 - 1) + (0,1,1)
                state[index_state_0] = 1/2**0.5
                state[index_state_1] = 1/2**0.5 
            t_scr = int(scram_depth*L)
            T = 4*L

            # get data
            seed_outcome = np.random.randint(0,1000000000,1)
            state,entropy,correlation,_ = get_data_ancilla(state,L,L_A,T,t_scr,theta*np.pi/2,scrambling_type=scrambling_type,evolution_type=evolution_type,seed_unitary=seed_unitary,seed_scram=seed_scrambling,seed_outcomes=seed_outcome,BC=BC)
            entropy_data.append(entropy)
            correlation_data.append(correlation)
            
        data = {'entropy':entropy_data,
                'correlation':correlation_data,
                'seed_outcome': seed_outcome,
                'seed_unitary':seed_unitary,
                'seed_scrambling':seed_scrambling}
                          
        #save data
        file_dir = get_filename(t_scr=t_scr,scram_U_type=scrambling_type,evolution_U_type=evolution_type,root_direc=root_direc)
        file_dir = file_dir + '/L='+str(L)
        file_dir = file_dir+'/T='+str(T)+'_tscr='+str(t_scr)+'_theta='+str(theta)+'_BC='+BC
        if os.path.isdir(file_dir):
            shutil.rmtree(file_dir)
        if not os.path.isdir(file_dir):
            os.makedirs(file_dir)
            
        filename = file_dir+'/shots='+str(shots)+'_'+str(np.random.randint(0,1000000000,1))
        
        with open(filename,'wb') as f:
            pickle.dump(data,f)
        
        print("L={}, theta={}, time={}".format(L,theta,time.time()-start))

L=6, theta=0.1, time=32.38157606124878
L=8, theta=0.1, time=38.43791389465332
L=10, theta=0.1, time=1018.4537472724915
L=12, theta=0.1, time=6972.636835098267
L=14, theta=0.1, time=21410.49255490303
L=6, theta=0.2, time=928.8766100406647
L=8, theta=0.2, time=2012.887601852417
L=10, theta=0.2, time=1536.8037598133087
L=12, theta=0.2, time=3323.0531599521637
L=14, theta=0.2, time=722.3337407112122
L=6, theta=0.3, time=29.74330496788025
L=8, theta=0.3, time=32.72073006629944
L=10, theta=0.3, time=42.84312129020691
L=12, theta=0.3, time=85.74325394630432
L=14, theta=0.3, time=347.8409478664398
L=6, theta=0.4, time=28.47870898246765
L=8, theta=0.4, time=31.92964506149292
L=10, theta=0.4, time=41.7631630897522
L=12, theta=0.4, time=84.08234214782715
L=14, theta=0.4, time=326.0016713142395
L=6, theta=0.5, time=28.301448106765747
L=8, theta=0.5, time=31.52537703514099
L=10, theta=0.5, time=41.82712388038635
L=12, theta=0.5, time=83.85689115524292
L=14, theta=0.5, time=314.69184017181396
L=6, t

In [271]:
scrambling_type = 'haar'
evolution_type = 'haar'
def read_entropy(L,T,t_scr,theta,BC):
    file_dir = get_filename(t_scr=t_scr,scram_U_type=scrambling_type,evolution_U_type=evolution_type,root_direc=root_direc)
    file_dir = file_dir + '/L='+str(L)
    file_dir = file_dir+'/T='+str(T)+'_tscr='+str(t_scr)+'_theta='+str(theta)+'_BC='+BC
    for filename in os.listdir(file_dir):
        with open(file_dir + '/' + filename,'rb') as f:
            data = pickle.load(f)
        entropy_list = data['entropy']
        entropy = np.array(entropy_list)
    return entropy

In [274]:
def get_slope(xdata,ydata):
    zz = np.polyfit(xdata,ydata,1,full=True)
    p = np.poly1d(zz[0])
    return zz[0], p

slope = []
L_list = [6,8,10,12,14]
for L in [6,8,10,12,14][1:]:
    T = 4*L
    t_scr = L
    theta = 0.4
    entropy = read_entropy(L,T,t_scr,theta,BC=BC)
    ydata = np.average(entropy,axis=0)
    err = np.std(entropy,axis=0)/(entropy.shape[0]-1)**0.5
    xdata = np.arange(0,T,1)
    zz,p = get_slope(xdata[L:],np.log(ydata[L:,0]))
    slope.append(-zz[0])
    # print(entropy[-1])
    pl.errorbar(xdata[:]/L,ydata[:,0],yerr=err[:,0],ls='-',marker='o',label='L={}'.format(L))
    pl.plot(xdata[:]/L,np.exp(p(np.array(xdata))),'k',ls=':')

pl.legend()

In [261]:
pl.plot(L_list[1:],slope,'-o')